In [56]:
%pwd

'/Users/waittim/Documents/Python/Personal/mask-detector'

In [2]:
%cd ..

/Users/waittim/Documents/Python/Personal/mask-detector


In [ ]:
%pip install -r requirements.txt

In [48]:
import os
import sys
import xml.etree.ElementTree as ET
import glob
from PIL import Image
import random

In [63]:
def xml_to_txt(indir, outdir, imagedir):
 
    #os.chdir(indir)
    annotations = os.listdir(indir+'/.')
    #annotations = glob.glob(str(annotations)+'*.xml')
    #print(annotations)
    for i, file in enumerate(annotations):
         
        file_save = file.split('.')[0]+'.txt'
        #print(file_save)
        # file_txt=os.path.join(outdir,file_save)
        file_txt = outdir+"/"+file_save
        f_w = open(file_txt, 'w+')
 
        # actual parsing
        in_file = open(indir+'/'+file)
        tree=ET.parse(in_file)
        root = tree.getroot()
        filename = root.find('filename').text  #Get filename from the root of xml
        
        # Get the image size
        image = Image.open(imagedir+'/'+file.split('.')[0]+'.jpg')
        image_width = image.width
        image_height = image.height
        
        for obj in root.iter('object'):
                current = list()
                name = obj.find('name').text   #get the class name then transfer to number
                if name == 'face':
                    class_number = '0'
                elif name == 'face_mask':
                    class_number = '1'
                else: 
                    class_number = '2' #wrong class
                    print(file_save) # if return any wrong class, get correct it
 
                xmlbox = obj.find('bndbox')
                xmin = int(xmlbox.find('xmin').text)
                xmax = int(xmlbox.find('xmax').text)
                ymin = int(xmlbox.find('ymin').text)
                ymax = int(xmlbox.find('ymax').text)
                
                x_center = str((xmin + xmax) / 2 / image_width)
                y_center = str((ymin + ymax) / 2 / image_height)
                width = str((xmax - xmin) / image_width)
                height = str((ymax - ymin) / image_height)
                
                #print xn
                f_w.write(class_number +' '+x_center+' '+y_center+' '+width+' '+height+'\n')
 
 

In [64]:
indir = './modeling/data/xml_labels'   #xml fold
outdir = './modeling/data/labels'  #txt fold
imagedir = './modeling/data/images'  # image fold

In [65]:
xml_to_txt(indir,outdir,imagedir)

In [71]:
test_percentage = 0.998

all_image = os.listdir(imagedir+'/.')

test_index = list(random.sample(range(0,len(all_image)), round(len(all_image)*test_percentage)))
train_index = [i for i in range(0,len(all_image)) if i not in test_index]

test_data = [all_image[i] for i in test_index]
train_data = [all_image[i] for i in train_index]

In [72]:
f_w = open('./modeling/data/train.txt', 'w+')
for i in train_data:
    f_w.write('./modeling/data/images/'+i+'\n')
f_w.close()

f_w = open('./modeling/data/valid.txt', 'w+')
for i in test_data:
    f_w.write('./modeling/data/images/'+i+'\n')
f_w.close()

In [ ]:
f_w = open('./modeling/data/face_mask.names.txt', 'w+')
class_list = ['face', 'face_mask']
for i in class_list:
    f_w.write(i+'\n')

f_w.close()
    
os.rename('./modeling/data/face_mask.names.txt', './modeling/data/face_mask.names')

In [95]:
f_w = open('./modeling/data/face_mask.data.txt', 'w+')
content = ['classes=2', 
           'train=data/train.txt', 
           'valid=data/valid.txt', 
           'names=data/face_mask.names']
for i in content:
    f_w.write(i+'\n')

f_w.close()
    
os.rename('./modeling/data/face_mask.data.txt', './modeling/data/face_mask.data')